In [ ]:
#Importing the necessary libraries

import os 
import sys
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import ticker
import numpy as np

import statsmodels.api as sm


In [ ]:
#Global Settings 
pd.set_option('display.float_format', lambda x: '%.4f' % x)

In [ ]:
#Importing Configs
# Define the path where config.py is located
#Mac
os.chdir('/Users/kylenabors/Documents/GitHub/Finance-ML-Modeling')
config_file_path = os.getcwd()
print(config_file_path)

# Add this path to the sys.path
sys.path.append(config_file_path)

import config

#Configs
finbert_models = config.finbert_models
Body = config.Body
database = config.Local_Database

In [ ]:
sentiment = pd.date_range(start='1/1/1990', end='1/1/2024', freq='D')
sentiment = pd.DataFrame(sentiment, columns=['date'])

state = pd.read_csv(f"{finbert_models}/{Body}/Statements/{Body}_Statements_finbert_model_short.csv")
state = state[['date','sentiment']]
state = state.rename(columns={'sentiment':'statement_sentiment'})
state['date'] = pd.to_datetime(state['date'])
sentiment = pd.merge(sentiment, state, how='left', left_on='date', right_on='date')

minutes = pd.read_csv(f"{finbert_models}/{Body}/Minutes/{Body}_Minutes_finbert_model_short.csv")
minutes = minutes[['date','sentiment']]
minutes = minutes.rename(columns={'sentiment':'minute_sentiment'})
minutes['date'] = pd.to_datetime(minutes['date'])
sentiment = pd.merge(sentiment, minutes, how='left', left_on='date', right_on='date')

Body = 'ECB'
mpd = pd.read_csv(f"{finbert_models}/{Body}/Monetary policy decisions/{Body}_Monetary policy decisions_finbert_model_short.csv")
mpd = mpd[['date','sentiment']]
mpd = mpd.rename(columns={'sentiment':'mpd_sentiment'})
mpd['date'] = pd.to_datetime(mpd['date'])
sentiment = pd.merge(sentiment, mpd, how='left', left_on='date', right_on='date')

fed = pd.merge(state, minutes, on='date', how='outer')
fed = fed.groupby('date').mean().reset_index()
fed['fed_sentiment'] = fed[['statement_sentiment', 'minute_sentiment']].mean(axis=1)
fed = fed[['date', 'fed_sentiment']]    
sentiment = pd.merge(sentiment, fed, how='outer', left_on='date', right_on='date')

In [ ]:
market =pd.read_csv(f'{database}/Market Data/All Market Data.csv')
market = market.rename(columns={'PCE':'pce', 'GDP':'gdp', 'GDPPOT':'gdp_pot'})
market['date'] = pd.to_datetime(market['date'])
print(market.columns)


In [ ]:
taylor = pd.read_csv(f'/Users/kylenabors/Documents/Database/Market Data/Taylor/Taylor.csv')
taylor['date'] = pd.to_datetime(taylor['date'])
taylor = taylor.rename(columns={'Taylor':'taylor_fred'})

In [ ]:

sentiment = pd.merge(sentiment, market, how='left', left_on='date', right_on='date')
sentiment = pd.merge(sentiment, taylor, how='left', left_on='date', right_on='date')

In [ ]:
sentiment = sentiment[(sentiment['date'] >= '2000-01-01') & (sentiment['date'] <= '2023-06-30')]
sentiment = sentiment.set_index('date')
sentiment = sentiment.resample('Q').mean().reset_index()


In [ ]:

filter_items = ['pce', 'gdp', 'gdp_pot', 'inflation', 'taylor_fred']

for item in filter_items:
    filter_df = sentiment.copy(deep=True)
    filter_df = filter_df[['date', f'{item}']]

    cycle, trend = sm.tsa.filters.hpfilter(filter_df[f'{item}'], 1600)

    item_decomp = filter_df[[f'{item}']]
    item_decomp[f"{item}_cycle"] = cycle
    item_decomp[f"{item}_trend"] = trend

    filter_df = pd.merge(filter_df, item_decomp, on=f'{item}', how='left')  
    filter_df = filter_df[['date', f'{item}_cycle', f'{item}_trend']]
    sentiment = pd.merge(sentiment, filter_df, on='date', how='left')

In [ ]:

sentiment['gap'] = (sentiment['gdp'] - sentiment['gdp_pot'])/sentiment['gdp_pot']
sentiment['gap'] = sentiment['gap']*100

In [ ]:
sentiment['taylor'] = 2 + sentiment['gdpdef'] + 0.5*(sentiment['gdpdef'] - 2) + 0.5*(sentiment['gap'])


In [ ]:
#if statment_sentiment is greater than 0 make 1 if it is less than 0 make -1
#sentiment['statement_sentiment'] = np.where(sentiment['statement_sentiment'] > 0, 1, -1)

In [ ]:

sentiment_groups = ['statement', 'minute', 'fed', 'mpd']
for group in sentiment_groups:
    for i in range(0, 11):
        sentiment[f'{group}_sentiment_{i}'] = sentiment[f'{group}_sentiment'].shift(i)
        sentiment[f'{group}_sentiment_forward_{i}'] = sentiment[f'{group}_sentiment'].shift(-i)
        
market_groups = ['fedfunds', 'taylor']

for group in market_groups:
    for i in range(0, 11):
        sentiment[f'{group}_{i}'] = sentiment[f'{group}'].shift(i)
        sentiment[f'{group}_forward_{i}'] = sentiment[f'{group}'].shift(-i)

In [ ]:
#Caculate correlation of All Minutes sentiment with taylor rule
correlation = sentiment[['minute_sentiment','minute_sentiment_1', 'minute_sentiment_2', 'minute_sentiment_3', 'minute_sentiment_4', 'minute_sentiment_5', 'minute_sentiment_6', 'minute_sentiment_7', 'minute_sentiment_8', 'minute_sentiment_9', 'taylor_fred', 'fedfunds']].corr()
print(correlation)

In [ ]:

sentiment.to_csv(f'/Users/kylenabors/Documents/Database/Models/FinBERT Models/taylor.csv', index=False)


In [ ]:
sentiment['statement_sentiment'] = (sentiment['statement_sentiment'] - sentiment['statement_sentiment'].mean()) / (sentiment['statement_sentiment'].max() - sentiment['statement_sentiment'].min())
sentiment['minute_sentiment'] = (sentiment['minute_sentiment'] - sentiment['minute_sentiment'].mean()) / (sentiment['minute_sentiment'].max() - sentiment['minute_sentiment'].min())


sentiment['taylor'] = (sentiment['taylor'] - sentiment['taylor'].mean()) / (sentiment['taylor'].max() - sentiment['taylor'].min())
sentiment['fedfunds'] = (sentiment['fedfunds'] - sentiment['fedfunds'].mean()) / (sentiment['fedfunds'].max() - sentiment['fedfunds'].min())
sentiment['sp500_price'] = (sentiment['sp500_price'] - sentiment['sp500_price'].mean()) / (sentiment['sp500_price'].max() - sentiment['sp500_price'].min())
sentiment['vix_sentiment'] = (sentiment['vix_sentiment'] - sentiment['vix_sentiment'].mean()) / (sentiment['vix_sentiment'].max() - sentiment['sp500_price'].min())

In [ ]:
print(sentiment.columns)

#Graph the Data
fig, ax = plt.subplots(figsize = (40, 20))

ax.scatter(sentiment['date'], sentiment['statement_sentiment'], label='FOMC Statement Sentiment', color='blue')
ax.plot(sentiment['date'], sentiment['statement_sentiment'], label='FOMC Statement Sentiment', color='lightblue')

ax.scatter(sentiment['date'], sentiment['minute_sentiment'], label='FOMC Minute Sentiment', color='green')
ax.plot(sentiment['date'], sentiment['minute_sentiment'], label='FOMC Minute Sentiment', color='lightgreen')

ax.scatter(sentiment['date'], sentiment['taylor'], label='Taylor Rule', color='red')
ax.plot(sentiment['date'], sentiment['taylor'], label='Taylor Rule', color='lightcoral')

# ax.scatter(sentiment['date'], sentiment['fedfunds'], label='Fedfunds', color='yellow')
# ax.plot(sentiment['date'], sentiment['fedfunds'], label='Fedfunds', color='yellow')

ax.scatter(sentiment['date'], sentiment['sp500_price'], label='SP500', color='black')
ax.plot(sentiment['date'], sentiment['sp500_price'], label='SP500', color='grey')

ax.scatter(sentiment['date'], sentiment['vix_sentiment'], label='VIX', color='purple')
ax.plot(sentiment['date'], sentiment['vix_sentiment'], label='VIX', color='violet')

#Create a vertical line at every year 
for year in range(2000, 2025):
    ax.axvline(pd.to_datetime(f'{year}-01-01'), color='gray', linestyle='--', lw=1)

ax.set_xlabel('Date')
ax.set_ylabel('Taylor Rule')
ax.legend()
plt.show()


In [ ]:
import plotly.graph_objs as go
from plotly.subplots import make_subplots

# Assuming 'sentiment' is your DataFrame and it has been properly preprocessed

# Create subplots
fig = make_subplots(rows=1, cols=1)

# Add traces
fig.add_trace(go.Scatter(x=sentiment['date'], y=sentiment['statement_sentiment'], mode='lines', name='FOMC Statement Sentiment'))
fig.add_trace(go.Scatter(x=sentiment['date'], y=sentiment['minute_sentiment'], mode='lines', name='FOMC Minute Sentiment'))
fig.add_trace(go.Scatter(x=sentiment['date'], y=sentiment['taylor'], mode='lines', name='Taylor Rule'))
fig.add_trace(go.Scatter(x=sentiment['date'], y=sentiment['fedfunds'], mode='lines', name='fedfunds'))
fig.add_trace(go.Scatter(x=sentiment['date'], y=sentiment['sp500_price'], mode='lines', name='SP500 Price'))
fig.add_trace(go.Scatter(x=sentiment['date'], y=sentiment['vix_sentiment'], mode='lines', name='VIX Sentiment'))

# Update layout for interactive functionality
fig.update_layout(
    title='Interactive Plot',
    xaxis_title='Date',
    yaxis_title='Value',
    showlegend=True
)

# Add vertical lines at each year start for reference (this is just an example, adjust as necessary)
for year in range(sentiment['date'].dt.year.min(), sentiment['date'].dt.year.max() + 1):
    fig.add_vline(x=f'{year}-01-01', line=dict(color='gray', dash='dash'))

# Show the interactive plot
fig.show()
#Export the plot as html
fig.write_html("/Users/kylenabors/Documents/Database/Models/FinBERT Models/interactive_plot.html")
